In [87]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

from neuropacks import ECOG
from pyuoi.decomposition import NMF as uNMF
from sklearn.cluster import DBSCAN
from sklearn.decomposition import NMF

%matplotlib inline

In [2]:
data_path = '/Users/psachdeva/data/ECoG/data/r32_b7.mat'
grid_path = '/Users/psachdeva/data/ECoG/data/grdid.mat'
ecog = ECOG(
    data_path=data_path,
    grid_path=grid_path
)

In [89]:
Y = ecog.get_response_matrix(
    bounds=(40, 60),
    band='HG'
)

In [90]:
# kill the dead indices
dead_indices = [19, 21, 0]
Y[:, dead_indices] = 0

In [91]:
# Y = Y + np.abs(np.min(Y, axis=0, keepdims=True))
Y[Y < 0] = 0

In [92]:
uoi = uNMF(
    n_boots=20,
    ranks=[20],
    dbscan=DBSCAN(min_samples=10),
    nmf=NMF(beta_loss='kullback-leibler', solver='mu', max_iter=1000, init='random'),
    cons_meth=np.median
)
uoi.fit(Y)

NMF(cons_meth=<function median at 0x109db4d90>,
  dbscan=DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=10, n_jobs=None, p=None),
  n_boots=20,
  nmf=NMF(alpha=0.0, beta_loss='kullback-leibler', init='random', l1_ratio=0.0,
  max_iter=1000, n_components=20, random_state=None, shuffle=False,
  solver='mu', tol=0.0001, verbose=0),
  nnreg=<function NMF.__initialize.<locals>.<lambda> at 0x1c1b70f048>,
  random_state=None, ranks=(20,))

In [93]:
nmf = NMF(
    n_components=uoi.components_.shape[0],
    beta_loss='kullback-leibler',
    solver='mu',
    max_iter=1000,
    init='random'
)
nmf.fit(Y)

NMF(alpha=0.0, beta_loss='kullback-leibler', init='random', l1_ratio=0.0,
  max_iter=1000, n_components=15, random_state=None, shuffle=False,
  solver='mu', tol=0.0001, verbose=0)

In [101]:
fits = h5py.File('ecog_nmf', 'a')
fits['uoi_zeroed'] = uoi.components_
fits['nmf_zeroed'] = nmf.components_
fits.close()